In [122]:
import nltk
from nltk.corpus import wordnet
import random
from nltk.corpus import movie_reviews
from nltk.classify import NaiveBayesClassifier
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import LinearSVC,SVC,NuSVC
from nltk.classify import ClassifierI
from statistics import mode

In [39]:
nltk.download('wordnet')
nltk.download('movie_reviews')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

In [7]:
syns=wordnet.synsets("program")
print(syns)

[Synset('plan.n.01'), Synset('program.n.02'), Synset('broadcast.n.02'), Synset('platform.n.02'), Synset('program.n.05'), Synset('course_of_study.n.01'), Synset('program.n.07'), Synset('program.n.08'), Synset('program.v.01'), Synset('program.v.02')]


In [9]:
print(syns[0].lemmas())

[Lemma('plan.n.01.plan'), Lemma('plan.n.01.program'), Lemma('plan.n.01.programme')]


In [14]:
print(syns[0].definition())

a series of steps to be carried out or goals to be accomplished


In [17]:
print(syns[1].examples())

['he proposed an elaborate program of public works', 'working mothers rely on the day care program']


In [20]:
synonyms=[]
antonyms=[]
for syn in wordnet.synsets("good"):
  for l in syn.lemmas():
    print("l:",l)
    synonyms.append(l.name())
    if l.antonyms():
      antonyms.append(l.antonyms()[0].name())
#print(set(synonyms))
#print(set(antonyms))

l: Lemma('good.n.01.good')
l: Lemma('good.n.02.good')
l: Lemma('good.n.02.goodness')
l: Lemma('good.n.03.good')
l: Lemma('good.n.03.goodness')
l: Lemma('commodity.n.01.commodity')
l: Lemma('commodity.n.01.trade_good')
l: Lemma('commodity.n.01.good')
l: Lemma('good.a.01.good')
l: Lemma('full.s.06.full')
l: Lemma('full.s.06.good')
l: Lemma('good.a.03.good')
l: Lemma('estimable.s.02.estimable')
l: Lemma('estimable.s.02.good')
l: Lemma('estimable.s.02.honorable')
l: Lemma('estimable.s.02.respectable')
l: Lemma('beneficial.s.01.beneficial')
l: Lemma('beneficial.s.01.good')
l: Lemma('good.s.06.good')
l: Lemma('good.s.07.good')
l: Lemma('good.s.07.just')
l: Lemma('good.s.07.upright')
l: Lemma('adept.s.01.adept')
l: Lemma('adept.s.01.expert')
l: Lemma('adept.s.01.good')
l: Lemma('adept.s.01.practiced')
l: Lemma('adept.s.01.proficient')
l: Lemma('adept.s.01.skillful')
l: Lemma('adept.s.01.skilful')
l: Lemma('good.s.09.good')
l: Lemma('dear.s.02.dear')
l: Lemma('dear.s.02.good')
l: Lemma('dear.s

In [33]:
w1=wordnet.synset("Me.n.01")
w2=wordnet.synset("me.n.01")
print(w1.wup_similarity(w2))

1.0


In [47]:
#Text classifier for sentiment analysis
doc=[]
for category in movie_reviews.categories():
  for fileid in movie_reviews.fileids(category):
    doc.append((movie_reviews.words(fileid),category))
random.shuffle(doc)

In [48]:
print(doc[1])

(['let', "'", 's', 'say', 'you', 'live', 'at', 'the', ...], 'pos')


In [49]:
all_words=[]
for i in movie_reviews.words():
  all_words.append(i.lower())
all_words=nltk.FreqDist(all_words)
print(all_words.most_common(15))

[(',', 77717), ('the', 76529), ('.', 65876), ('a', 38106), ('and', 35576), ('of', 34123), ('to', 31937), ("'", 30585), ('is', 25195), ('in', 21822), ('s', 18513), ('"', 17612), ('it', 16107), ('that', 15924), ('-', 15595)]


In [61]:
wf=list(all_words.keys())[:3000]

In [62]:
def d_features(doc):
  words=set(doc)
  features={}
  for w in wf:
    features[w]=w in words
  return features

In [63]:
print(d_features(movie_reviews.words('neg/cv000_29416.txt')))

{'plot': True, ':': True, 'two': True, 'teen': True, 'couples': True, 'go': True, 'to': True, 'a': True, 'church': True, 'party': True, ',': True, 'drink': True, 'and': True, 'then': True, 'drive': True, '.': True, 'they': True, 'get': True, 'into': True, 'an': True, 'accident': True, 'one': True, 'of': True, 'the': True, 'guys': True, 'dies': True, 'but': True, 'his': True, 'girlfriend': True, 'continues': True, 'see': True, 'him': True, 'in': True, 'her': True, 'life': True, 'has': True, 'nightmares': True, 'what': True, "'": True, 's': True, 'deal': True, '?': True, 'watch': True, 'movie': True, '"': True, 'sorta': True, 'find': True, 'out': True, 'critique': True, 'mind': True, '-': True, 'fuck': True, 'for': True, 'generation': True, 'that': True, 'touches': True, 'on': True, 'very': True, 'cool': True, 'idea': True, 'presents': True, 'it': True, 'bad': True, 'package': True, 'which': True, 'is': True, 'makes': True, 'this': True, 'review': True, 'even': True, 'harder': True, 'wri

In [65]:
features_sets=[(d_features(rev),category) for (rev,category) in doc]

In [66]:
print(features_sets[0])

({'plot': False, ':': True, 'two': True, 'teen': False, 'couples': False, 'go': False, 'to': True, 'a': True, 'church': False, 'party': False, ',': True, 'drink': False, 'and': True, 'then': True, 'drive': False, '.': True, 'they': True, 'get': True, 'into': True, 'an': True, 'accident': False, 'one': True, 'of': True, 'the': True, 'guys': False, 'dies': False, 'but': True, 'his': True, 'girlfriend': False, 'continues': True, 'see': True, 'him': False, 'in': True, 'her': True, 'life': True, 'has': True, 'nightmares': False, 'what': False, "'": True, 's': True, 'deal': False, '?': True, 'watch': False, 'movie': True, '"': True, 'sorta': False, 'find': True, 'out': True, 'critique': False, 'mind': True, '-': True, 'fuck': False, 'for': True, 'generation': False, 'that': True, 'touches': False, 'on': True, 'very': True, 'cool': False, 'idea': False, 'presents': False, 'it': True, 'bad': True, 'package': False, 'which': True, 'is': True, 'makes': False, 'this': True, 'review': True, 'even'

In [67]:
train=features_sets[:1900]
test=features_sets[1900:]

In [70]:
classifier=NaiveBayesClassifier.train(train)

In [72]:
print(nltk.classify.accuracy(classifier,test)*100)

81.0


In [75]:
classifier.show_most_informative_features(15)

Most Informative Features
                   sucks = True              neg : pos    =     10.7 : 1.0
                  annual = True              pos : neg    =      9.6 : 1.0
                 frances = True              pos : neg    =      8.9 : 1.0
                  turkey = True              neg : pos    =      8.8 : 1.0
           unimaginative = True              neg : pos    =      8.4 : 1.0
                  sexist = True              neg : pos    =      7.7 : 1.0
                 idiotic = True              neg : pos    =      7.5 : 1.0
              schumacher = True              neg : pos    =      7.1 : 1.0
                  shoddy = True              neg : pos    =      7.1 : 1.0
             silverstone = True              neg : pos    =      7.1 : 1.0
                  regard = True              pos : neg    =      6.5 : 1.0
                  kombat = True              neg : pos    =      6.4 : 1.0
                    mena = True              neg : pos    =      6.4 : 1.0

In [99]:
MNB=SklearnClassifier(MultinomialNB())
MNB.train(train)
print(nltk.classify.accuracy(MNB,test)*100)


84.0


In [100]:
BNB=SklearnClassifier(BernoulliNB())
BNB.train(train)
print(nltk.classify.accuracy(BNB,test)*100)

82.0


In [112]:
LR=SklearnClassifier(LogisticRegression())
LR.train(train)
print(nltk.classify.accuracy(LR,test)*100)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  )


84.0


In [113]:
SGD=SklearnClassifier(SGDClassifier())
SGD.train(train)
print(nltk.classify.accuracy(SGD,test)*100)

81.0


In [117]:
SVC=SklearnClassifier(SVC())
SVC.train(train)
print(nltk.classify.accuracy(SVC,test)*100)

78.0


In [118]:
LSVC=SklearnClassifier(LinearSVC())
LSVC.train(train)
print(nltk.classify.accuracy(LSVC,test)*100)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  Specifies the loss function. 'hinge' is the standard SVM loss


84.0


In [123]:
class VoteClassifier(ClassifierI):
  def __init__(self, *classifiers):
    self. classifiers = classifiers
  def classify(self, features) :
    votes = []
    for c in self. classifiers:
      v = c.classify (features)
      votes. append (v)
      return mode (votes)
  def confidence (self, features) :
    votes = []
    for c in self. classifiers:
      v = c.classify (features)
      votes. append (v)
      return votes.count(mode(votes))/len(votes)